### Teste etapa 1

In [1]:
from tree import Tree
from utils import inputs

In [2]:
expression_trees = map(Tree.from_expression, inputs)

for idx, tree in enumerate(expression_trees):
    print(f'Arvore {idx+1} :')
    tree.bshow()
    print()

Arvore 1 :
┌───
|MEM|None|
│   ┌───|CONST 2|+|
└───|+|MEM|
    └───|CONST 1|+|

Arvore 2 :
    ┌───
┌───|MEM|MOVE|
│   │   ┌───|CONST x|+|
│   └───|+|MEM|
│       └───|FP|+|
|MOVE|None|
│   ┌───
└───|MEM|MOVE|
    │       ┌───|CONST 4|*|
    │   ┌───|*|+|
    │   │   └───|TEMP i|*|
    └───|+|MEM|
        │   ┌───
        └───|MEM|+|
            │   ┌───|CONST a|+|
            └───|+|MEM|
                └───|FP|+|



### Fase de seleção de instrução

In [3]:
jouette_architecture = {
    **{
        ins: {
            'patterns': [Tree(op)],
            'instruction': f'ri <- rj {op} rk'
        }  
        for ins, op in [('ADD', '+'), ('MUL', '*'), ('SUB', '-'), ('DIV', '/')]
    },
    'ADDI': {
        'patterns': [
            Tree('+', right='CONST'),
            Tree('+', left='CONST'),
            Tree('CONST')
        ],
        'instruction': 'ri <- rj + c' 
    },
    'SUBI': {
        'patterns': [
            Tree('-', right='CONST')
        ],
        'instruction': 'ri <- rj - c'
    },
    'LOAD': {
        'patterns': [
            Tree('MEM', left=Tree('+', right='CONST')),
            Tree('MEM', left=Tree('+', left='CONST')),
            Tree('MEM', left='CONST'),
            Tree('MEM')
        ],
        'instruction': 'ri <- M[ rj + c ]'
    },
    'STORE': {
        'patterns': [
            Tree('MOVE', left=Tree('MEM', left=Tree('+', right='CONST'))),
            Tree('MOVE', left=Tree('MEM', left=Tree('+', left='CONST'))),
            Tree('MOVE', left=Tree('MEM', left='CONST')),
            Tree('MOVE', left='MEM')
        ],
        'instruction': 'M[ rj + c ] <- ri'
    },
    'MOVEM': {
        'patterns': [
            Tree('MOVE', left='MEM', right='MEM')
        ],
        'instruction': 'M[ rj ] <- M[ ri ]'
    } 
}

In [4]:
for key, value in jouette_architecture.items():
    print(key)
    print('-'*len(key))
    for pattern in value['patterns']:
        pattern.bshow()
    print(f"instruction: {value['instruction']}")
    print('-'*len(key))
    print('-'*len(key))
    print()


ADD
---
|+|None|
instruction: ri <- rj + rk
---
---

MUL
---
|*|None|
instruction: ri <- rj * rk
---
---

SUB
---
|-|None|
instruction: ri <- rj - rk
---
---

DIV
---
|/|None|
instruction: ri <- rj / rk
---
---

ADDI
----
┌───|CONST|+|
|+|None|
└───
┌───
|+|None|
└───|CONST|+|
|CONST|None|
instruction: ri <- rj + c
----
----

SUBI
----
┌───|CONST|-|
|-|None|
└───
instruction: ri <- rj - c
----
----

LOAD
----
┌───
|MEM|None|
│   ┌───|CONST|+|
└───|+|MEM|
    └───
┌───
|MEM|None|
│   ┌───
└───|+|MEM|
    └───|CONST|+|
┌───
|MEM|None|
└───|CONST|MEM|
|MEM|None|
instruction: ri <- M[ rj + c ]
----
----

STORE
-----
┌───
|MOVE|None|
│   ┌───
└───|MEM|MOVE|
    │   ┌───|CONST|+|
    └───|+|MEM|
        └───
┌───
|MOVE|None|
│   ┌───
└───|MEM|MOVE|
    │   ┌───
    └───|+|MEM|
        └───|CONST|+|
┌───
|MOVE|None|
│   ┌───
└───|MEM|MOVE|
    └───|CONST|MEM|
┌───
|MOVE|None|
└───|MEM|MOVE|
instruction: M[ rj + c ] <- ri
-----
-----

MOVEM
-----
┌───|MEM|MOVE|
|MOVE|None|
└───|MEM|MOVE|
instr

In [5]:
jouette_architecture.keys()

dict_keys(['ADD', 'MUL', 'SUB', 'DIV', 'ADDI', 'SUBI', 'LOAD', 'STORE', 'MOVEM'])

In [6]:
costs = [
    {
        'instruction': 'TEMP',
        'cost': 0
    },
    {
        'instruction': 'MOVEM',
        'cost': 2
    }
]
other_costs = map(
    lambda ins: {'instruction': ins, 'cost':1},
    (set(jouette_architecture.keys()) - {'TEMP', 'MOVEM'})
)
costs = [*costs, *other_costs]

In [7]:
costs

[{'instruction': 'TEMP', 'cost': 0},
 {'instruction': 'MOVEM', 'cost': 2},
 {'instruction': 'ADDI', 'cost': 1},
 {'instruction': 'ADD', 'cost': 1},
 {'instruction': 'SUB', 'cost': 1},
 {'instruction': 'MUL', 'cost': 1},
 {'instruction': 'LOAD', 'cost': 1},
 {'instruction': 'DIV', 'cost': 1},
 {'instruction': 'SUBI', 'cost': 1},
 {'instruction': 'STORE', 'cost': 1}]

In [8]:
exp = inputs[-1]
exp

'MOVE(MEM(+(MEM(+(FP,CONST a)),*(TEMP i,CONST 4))),MEM(+(FP,CONST x)))'

In [9]:
exp_tree = Tree.from_expression(exp, verbose=True)
exp_tree.bshow()

    ┌───
┌───|MEM|MOVE|
│   │   ┌───|CONST x|+|
│   └───|+|MEM|
│       └───|FP|+|
|MOVE|None|
│   ┌───
└───|MEM|MOVE|
    │       ┌───|CONST 4|*|
    │   ┌───|*|+|
    │   │   └───|TEMP i|*|
    └───|+|MEM|
        │   ┌───
        └───|MEM|+|
            │   ┌───|CONST a|+|
            └───|+|MEM|
                └───|FP|+|


In [10]:
for leave in exp_tree.leaves():
    leave.bshow()

|FP|+|
|CONST a|+|
|TEMP i|*|
|CONST 4|*|
|FP|+|
|CONST x|+|


In [11]:
for leave in exp_tree.subtrees():
    leave.bshow()

┌───
|MEM|MOVE|
│       ┌───|CONST 4|*|
│   ┌───|*|+|
│   │   └───|TEMP i|*|
└───|+|MEM|
    │   ┌───
    └───|MEM|+|
        │   ┌───|CONST a|+|
        └───|+|MEM|
            └───|FP|+|
┌───
|MEM|MOVE|
│   ┌───|CONST x|+|
└───|+|MEM|
    └───|FP|+|


In [19]:
sub_tree = exp_tree.subtrees()[1]
sub_tree.bshow()

┌───
|MEM|MOVE|
│   ┌───|CONST x|+|
└───|+|MEM|
    └───|FP|+|


In [23]:
len(sub_tree.left.left)

1

In [12]:
len(exp_tree)

6

In [24]:
exp_tree.bshow()

    ┌───
┌───|MEM|MOVE|
│   │   ┌───|CONST x|+|
│   └───|+|MEM|
│       └───|FP|+|
|MOVE|None|
│   ┌───
└───|MEM|MOVE|
    │       ┌───|CONST 4|*|
    │   ┌───|*|+|
    │   │   └───|TEMP i|*|
    └───|+|MEM|
        │   ┌───
        └───|MEM|+|
            │   ┌───|CONST a|+|
            └───|+|MEM|
                └───|FP|+|


In [27]:
for l in exp_tree.leaves():
    l.bshow()

|FP|+|
|CONST a|+|
|TEMP i|*|
|CONST 4|*|
|FP|+|
|CONST x|+|


### Selector

In [51]:
from typing import Iterable
class Selector():

    PATTERN_INSTRUCTIONS = {
        1: "TEMP r{i}",
        2: "ADD r{i} <- r{j} + r{k}",
        3: "MUL r{i} <- r{j} * r{k}",
        4: "SUB r{i} <- r{j} - r{k}",
        5: "DIV r{i} <- r{j} / r{k}",
        6: "ADDI r{i} <- r{j} + {c}",
        7: "ADDI r{i} <- r{j} + {c}",
        8: "ADDI r{i} <- r{j} + {c}",
        9: "SUBI r{i} <- r{j} - {c}",
        10: "LOAD r{i} <- M[r{j} + {c}]",
        11: "LOAD r{i} <- M[r{j} + {c}]",
        12: "LOAD r{i} <- M[r{j} + {c}]",
        13: "LOAD r{i} <- M[r{j} + {c}]",
        14: "STORE M[r{j} + {c}] <- r{i}",
        15: "STORE M[r{j} + {c}] <- r{i}",
        16: "STORE M[r{j} + {c}] <- r{i}",
        17: "STORE M[r{j} + {c}] <- r{i}",
        18: "MOVEM M[r{j}] <- M[r{i}]",
    }

    
    def __init__(self, tree: Tree) -> None:
        self.tree: Tree = tree
        self.i = 0
        self.leaves = tree.leaves()
        self.check_tree(self.leaves)


    def check_tree(self, level: Iterable[Tree]):
        if level:
            for node in level:
                print(node.value)
            print('---------------')
            self.getNewLayer(level)

    def getNewLayer(self, level):
        new_level = []
        for node in level:
            if(node._father is not None):
                if (node._father not in new_level):
                    new_level.append(node._father)

        self.check_tree(new_level)

In [52]:
exp_tree.bshow()

    ┌───
┌───|MEM|MOVE|
│   │   ┌───|CONST x|+|
│   └───|+|MEM|
│       └───|FP|+|
|MOVE|None|
│   ┌───
└───|MEM|MOVE|
    │       ┌───|CONST 4|*|
    │   ┌───|*|+|
    │   │   └───|TEMP i|*|
    └───|+|MEM|
        │   ┌───
        └───|MEM|+|
            │   ┌───|CONST a|+|
            └───|+|MEM|
                └───|FP|+|


In [53]:
Selector(exp_tree)

FP
CONST a
TEMP i
CONST 4
FP
CONST x
---------------
+
*
+
---------------
MEM
+
MEM
---------------
+
MEM
MOVE
---------------
MEM
MOVE
---------------
MOVE
---------------
